In [1]:
from data.graph_loader import GraphLoader
from data.gtfs_loader import GTFSLoader
from core.routing import RouteService
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx

def openMap(m):
    html = "map.html"
    m.save(html)

    import webbrowser
    webbrowser.open(html)

In [2]:
gtfs_loader = GTFSLoader()
transit_df = gtfs_loader.load_transit_dataframe("data/gtfs")
stops_df = gtfs_loader.load_stops_dataframe("data/gtfs", transit_df)



Loading transit DataFrame from data/gtfs\transit_df.pkl
Loading stops DataFrame from data/gtfs\stops_df.pkl


In [3]:
graph_loader = GraphLoader()
graph_walk = graph_loader.create_graph_walk("data/graphs/ZMG_walk.pkl", "data/osm/ZMG_enclosure_2km.geojson")
graph_transit = graph_loader.create_graph_transit("data/graphs/ZMG_transit.pkl", stops_df)

Loading walking graph from data/graphs/ZMG_walk.pkl
Loading transit graph from data/graphs/ZMG_transit_300.pkl
Loading transit graph from data/graphs/ZMG_transit_300.pkl


In [4]:
route_service = RouteService(graph_walk, graph_transit, stops_df, transit_df)

In [18]:
import random as rd
from shapely.geometry import Point

#choose randomly a node from the largest connected component
def random_conected_walking_nodes():

    connected = False
    attempts = 0

    while not connected:
        # Pick from largest connected component
        start = rd.choice(list(graph_walk.nodes))
        destination = rd.choice(list(graph_walk.nodes))
        
        # Since they're both in the same connected component, they should be connected
        connected = nx.has_path(graph_walk, source=start, target=destination)
        attempts += 1
        print(f"Attempt {attempts}")
        if attempts > 100:  # Much fewer attempts needed now
            raise Exception("Failed to find connected nodes after 100 attempts")
            
    return start, destination

walk_node_start, walk_node_destination = random_conected_walking_nodes()

Attempt 1


In [21]:
import folium

start = Point(graph_walk.nodes[walk_node_start]['x'], graph_walk.nodes[walk_node_start]['y'])
destination = Point(graph_walk.nodes[walk_node_destination]['x'], graph_walk.nodes[walk_node_destination]['y'])

df, total_time, m_comb = route_service.route_combined(start, destination)
# Add layer control
folium.LayerControl().add_to(m_comb)

openMap(m_comb)



      mode    shape_id                                    route_long_name  \
0  walking        None                                               None   
1  transit   C41-V2_r1  Troncal C41-V2 - 619 "Amarilla Cantaros - Cent...   
2  walking        None                                               None   
3  transit   MP-T01_r1                            Mi Macro Periférico T01   
4  walking        None                                               None   
5  transit  C110-V2_r2      Troncal C110-V2 - 629B "Primavera - La Venta"   

  route_short_name          trip_headsign route_type route_color  \
0             None                   None    walking     #333333   
1           C41-V2   Central de Autobuses          3     #4C8D2B   
2             None                   None    walking     #333333   
3           MP-T01  Barranca De Huentitán          1     #513780   
4             None                   None    walking     #333333   
5          C110-V2           La Primavera          3

In [25]:
# Add route segments to the map one by one with styling and stop markers
import folium
from shapely.geometry import LineString
from pyproj import Transformer

# Base map from previous cell: m_comb
#create empty map 
m = folium.Map(location=[20.6597, -103.3496], zoom_start=12)  # Example location (Guadalajara)

# Ensure transformer to 4326 for folium
_to4326 = Transformer.from_crs(3857, 4326, always_xy=True)

# Helper to compute weight from route_type
# Walking -> thinnest; then types 3,2,1; type 0 is thickest
# If route_type is None for walking, set minimal
def weight_for_route_type(route_type):
    if route_type == 'walking' or route_type is None:
        return 3
    # Numeric mapping: bigger width for smaller type value
    mapping = {0: 12, 1: 9, 2: 7, 3: 5}
    return mapping.get(route_type, 6)

# Iterate rows in df and add to map
for _, row in df.iterrows():
    mode = row.get('mode')
    color = row.get('route_color', '#3366cc')
    seg_geom = row.get('segment_geometry')
    route_type = row.get('route_type')
    long_name = row.get('route_long_name')
    short_name = row.get('route_short_name')
    headsign = row.get('trip_headsign')
    seconds = row.get('segment_time_seconds')

    # Build tooltip text
    tooltip_txt = []
    tooltip_txt.append(f"Modo: {mode}")
    if long_name or short_name:
        tooltip_txt.append(f"Ruta: {long_name or ''} {('('+short_name+')') if short_name else ''}")
    if seconds is not None and not pd.isna(seconds):
        tooltip_txt.append(f"Tiempo segmento: {int(seconds)} s")
    if headsign:
        tooltip_txt.append(f"Dirección: {headsign}")
    tooltip_html = "<br>".join(tooltip_txt)

    # Draw line if geometry exists
    if isinstance(seg_geom, LineString):
        coords3857 = list(seg_geom.coords)
        coords4326 = [[_to4326.transform(x, y)[1], _to4326.transform(x, y)[0]] for (x, y) in coords3857]
        folium.PolyLine(
            locations=coords4326,
            color=color,
            weight=weight_for_route_type(route_type),
            opacity=0.9,
            tooltip=tooltip_html
        ).add_to(m)

    # Add stop circles
    stops = row.get('stops', [])
    stop_names = row.get('stop_names', [])
    stop_positions = row.get('stop_positions', [])
    # Expect stop_positions as list of dicts with lat/lon in 4326
    for i, pos in enumerate(stop_positions):
        lat = pos.get('lat')
        lon = pos.get('lon')
        if lat is None or lon is None:
            continue
        # Style: transit mode first/last are white fill with black border; others use route color
        if mode == 'transit' and (i == 0 or i == len(stop_positions)-1):
            fill_color = 'white'
            line_color = 'black'
            radius = 8
        else:
            fill_color = color
            line_color = color
            radius = 6
        # Tooltip prefix: Estación for route_type 0 or 1, otherwise Parada
        prefix = 'Estación' if route_type in [0, 1] else 'Parada'
        folium.CircleMarker(
            location=[lat, lon],
            radius=radius,
            color=line_color,
            fill=True,
            fill_color=fill_color,
            fill_opacity=1.0,
            weight=2,
            tooltip=f"{prefix}: {stop_names[i] if i < len(stop_names) else ''}"
        ).add_to(m)


openMap(m)